# Clustering of imaging data

In [1]:
import numpy as np                     # scientific computing toolkit
import pandas as pd                    # data analysis toolkit
import scanpy as sc                    # scanpy is referred to with sc.***
import matplotlib.pyplot as plt        # Matplotlib is referred to with plt.***
from sklearn import preprocessing
from scipy import stats
import seaborn as sns
import math

sc.settings.verbosity = 0              # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()            # check if all needed versions are installed and up to date

scanpy==1.4.6 anndata==0.7.1 umap==0.3.10 numpy==1.18.1 scipy==1.4.1 pandas==0.25.3 scikit-learn==0.22.2 statsmodels==0.11.1 python-igraph==0.8.0


In [2]:
sc.settings.set_figure_params(dpi=80, color_map='viridis')

## Import and preprocessing:

#### Read the files for channels, FL_values and distances generated in ImageJ as pandas DataFrames:

In [3]:
data=pd.read_csv('./data/FL_values.csv',)
channels=pd.read_csv('./data/channels.csv')
gatings=pd.read_csv('./data/gatings.csv',sep=';')
gatings=gatings.fillna(0).astype(int)
gatings=gatings-1
gatings=gatings.astype(str)
cellnumber=int(data.shape[0]/channels.shape[0])

/Users/Sebastian/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Generate a matrix with the mean intensity values for each channel listed in additional columns and reshape the DataFrame:

In [4]:
matrix=pd.DataFrame(data[:cellnumber])
for i in range(0,channels.shape[0]):
    matrix[str(channels.iloc[i,0])]=0.000
    matrix[str(channels.iloc[i,0])][:cellnumber]=data.Mean[i*cellnumber:(i+1)*cellnumber]

/Users/Sebastian/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [5]:
#Drop columns which will not be used in the adata object and replace NaN with a 0
matrix.fillna(0,inplace=True)
matrix_new = matrix.drop(matrix.iloc[:, 0:35], axis=1)
matrix.rename(columns={'Circ.':'Circularity'}, inplace=True)
matrix_new = matrix_new[matrix_new.columns.drop(list(matrix_new.filter(regex='min_Dist')))]
matrix_normalized=matrix_new
matrix_normalized.index = pd.RangeIndex(len(matrix_normalized.index))

#Normalize the values within each marker 
for i in range(0,channels.shape[0]):
    df=matrix_new[str(channels.iloc[i,0])]
    matrix_normalized[str(channels.iloc[i,0])]=(df-df.min())/(df.max()-df.min())*100000

#export matrix to read in again as adata object
matrix_normalized.to_csv('./export/matrix.csv')

#### Read in matrix as scanpy object and save some variables from the initial matrix as adata.obs: